In [0]:
from google.colab import files
files.upload()

In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/


In [0]:
!kaggle competitions download -c dogs-vs-cats

In [0]:
import zipfile

In [0]:
with zipfile.ZipFile("/content/test1.zip","r") as zp:
  zp.extractall("content/test1")

In [0]:
with zipfile.ZipFile("/content/train.zip","r") as zp:
  zp.extractall("content/train")

In [0]:
import cv2
datadir='content/train/train'
#{'cat':0,'dog':1}
training_data=[]
import os
IMG_SIZE=50
for img in os.listdir(datadir):
  #print(img)
  q=img
  a=q.split('.')[0]
  if a=='cat':
    class_num=0
  else:
    class_num=1
  img_array=cv2.imread(os.path.join(datadir,img),cv2.IMREAD_GRAYSCALE)
  new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
  training_data.append([new_array,class_num])
  
print(len(training_data))


In [0]:
import random
import numpy as np
random.shuffle(training_data)
X=[]
y=[]
for features,label in training_data:
    X.append(features)
    y.append(label)
X=np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,1)

In [0]:
!pip install --upgrade tensorflow


In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.core import Dense,Activation,Flatten
from tensorflow.python.keras.layers.convolutional import Conv2D,MaxPooling2D
import pickle
from tensorflow.python.keras.callbacks import TensorBoard
import time
#NAME='CAT_vs_DOG_CNN_64x2-{}'.format(int(time.time()))
#gpu_option=tf.GPUoptions(per_process_gpu_memory_fraction=0.333)
#sess=tf.Session(config=tf.ConfigProto(gpu_options=gpu_option))
#X=pickle.load("x.pickle",'rd')
#y=pickle.load("y.pickle",'rd')
X=X/255.0
#optimize the model
dense_layers=[0,1,2]
layer_sizes=[32,64,128]
conv_layers=[1,2,3]
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME="{}-conv-{}-nodes-{}-dense-{}".format(conv_layer,layer_size,dense_layer,int(time.time()))
            tensorboard=TensorBoard(log_dir='logs_new/{}'.format(NAME))

            model=Sequential()
            model.add(Conv2D(layer_size,(3,3),input_shape=X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))
            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size,(3,3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))

            model.add(Flatten())
            for l in range(dense_layer):
                model.add(Dense(layer_size))
                #model.add(Activation("relu"))
            model.add(Dense(1))
            model.add(Activation('sigmoid'))
            model.compile(loss='binary_crossentropy',
                         optimizer='adam',
                         metrics=['accuracy'])
            model.fit(X,y,batch_size=32,epochs=3,validation_split=0.1,callbacks=[tensorboard])
            #at cmd type tensorboard --logdir=logs/ it will give the graphs.